In [1]:
from google.colab import drive #connecting google drive with this file
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pathlib
import random
import string
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

In [3]:
filePath = "/content/drive/MyDrive/Speech Conversion/ben2.txt" # path of the english-bangla file

text_pairs = []

for line in open(filePath):
    source, target = line.rstrip().split("\t")
    target = "[start] " + target + " [end]"
    text_pairs.append((source, target))

text_pairs[-5:]


[("I think it's highly unlikely that Tom will be allowed to keep the gold that he found.",
  '[start] আমার মনে হয় টম যে সোনা পেয়েছে সেটা তার কাছে রাখতে দেওয়া হবে এমন সম্ভাবনা খুব কম। [end]'),
 ("Tom told Mary that he was going to kill himself, but he didn't have the courage to do it.",
  '[start] টম মেরিকে বললো যে ও নিজেকে হত্যা করতে চলেছিলো, কিন্ত তা করার মতো সাহস ছিলো না। [end]'),
 ("Tom's an irritating person to work with because he'll never admit it when he's made a mistake.",
  '[start] টমের সঙ্গে কাজ করা খুব বিরক্তিকর কারণ ও কখনই মেনে নেয় না যে ও ভুল করেছে। [end]'),
 ("I thought doing this would be easy, but we've been working all day and we're still not finished.",
  '[start] আমি ভেবেছিলাম এটা করা সহজ হবে, কিন্তু আমরা সারাদিন ধরে কাজ করেছি আর এখনো শেষ করে উঠতে পারিনি। [end]'),
 ('January, February, March, April, May, June, July, August, September, October, November and December are the twelve months of the year.',
  '[start] বছরের বারোটা মাস হলো জানুয়ারি, ফেব্রুয়ারি, মার্চ

In [4]:
# spliting train,test and validation pairs
random.shuffle(text_pairs)
num_val_samples = int(0.05 * len(text_pairs)) # 0.05 means the 5% of data
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

4619 total pairs
4159 training pairs
230 validation pairs
230 test pairs


In [5]:
# this cell is for text vectorization

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


source_vectorization = TextVectorization(
    max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length,
)
target_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)

train_source_texts = [pair[0] for pair in train_pairs]
train_target_texts = [pair[1] for pair in train_pairs]

source_vectorization.adapt(train_source_texts)
target_vectorization.adapt(train_target_texts)

print(train_source_texts[:3],train_target_texts[:3])

['I have class tomorrow.', "I'd like you to talk to Tom.", 'Tom is afraid of the dark.'] ['[start] আমার কাল ক্লাস আছে। [end]', '[start] আমার ইচ্ছা যে তুই টমের সঙ্গে কথা বলিস। [end]', '[start] টম অন্ধকারকে ভয় পায়। [end]']


In [6]:

def format_dataset(source, target):
    source = source_vectorization(source)
    target = target_vectorization(target)
    return ({"encoder_inputs": source, "decoder_inputs": target[:, :-1],}, target[:, 1:])


def make_dataset(pairs):
    source_texts, target_texts = zip(*pairs)
    source_texts = list(source_texts)
    target_texts = list(target_texts)
    dataset = tf.data.Dataset.from_tensor_slices((source_texts, target_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [7]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


In [8]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "dense_dim": self.dense_dim,
            "num_heads": self.num_heads,
        })
        return config


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
    def get_config(self):
        config = super().get_config()
        config.update({
            "sequence_length": self.sequence_length,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "latent_dim": self.latent_dim,
            "num_heads": self.num_heads,
        })
        return config


In [11]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [10]:
epochs = 30

transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Epoch 1/30
65/65 [==============================] - 28s 241ms/step - loss: 5.7232 - accuracy: 0.3528 - val_loss: 4.7951 - val_accuracy: 0.3847
Epoch 2/30
65/65 [==============================] - 4s 60ms/step - loss: 4.5010 - accuracy: 0.3962 - val_loss: 4.3818 - val_accuracy: 0.4365
Epoch 3/30
65/65 [==============================] - 4s 63ms/step - loss: 3.9497 - accuracy: 0.4466 - val_loss: 3.9536 - val_accuracy: 0.4537
Epoch 4/30
65/65 [==============================] - 4s 64ms/step - loss: 3.4929 - accuracy: 0.4884 - val_loss: 3.6153 - val_accuracy: 0.4848
Epoch 5/30
65/65 [==============================] - 4s 63ms/step - loss: 3.0786 - accuracy: 0.5319 - val_loss: 3.3664 - val_accuracy: 0.5131
Epoch 6/30
65/65 [==============================] - 4s 64ms/step - loss: 2.7282 - accuracy: 0.5674 - val_loss: 3.2453 - val_accuracy: 0.5283
Epoch 7/30
65/65 [==============================] - 4s 63ms/step - loss: 2.3999 - accuracy: 0.6061 - val_loss: 3.0521 - val_accuracy: 0.5470
Epoch 8/30


In [ ]:
target_vocab = target_vectorization.get_vocabulary()
target_index_lookup = dict(zip(range(len(target_vocab)), target_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = target_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


predictions = []
references = [[]]
test_source_texts = [pair[0] for pair in test_pairs]
test_target_texts = [pair[1] for pair in test_pairs]

for _ in range(len(test_target_texts)):
    i = np.random.randint(0,len(test_source_texts)-1)
    input_sentence = test_source_texts[i]
    refs = test_target_texts[i]
    translated = decode_sequence(input_sentence)

    translated = translated.replace("[start]","")
    translated = translated.replace("[end]","")

    refs = refs.replace("[start]","")
    refs = refs.replace("[end]","")
    refs = [refs]

    references.append(refs)
    predictions.append(translated)

In [ ]:
references.pop(0)
print(len(references) , len(predictions))

443 443


In [ ]:
for i in range(10):
    input = test_source_texts[i]
    output = decode_sequence(input).replace("[start]","")
    output = output.replace("[end]","")
    print("input:" + input,"\toutput:" + output)

input:আমরা তোমার মন পড়েছি আর সব এখানেই আছে। 	output: আমি তর পুরিজ হিনিত্তে আর বেক ইয়োত আছে। 
input:কুকুরটা তোমার? 	output: তর ইক্কুনু 
input:কোথায় চলে গিয়েছিলে? 	output: হুদু জাগোই । 
input:যেরকম cc ক্যামেরা । যেটা হার্ড ড্রাইভে শুধু দিনের শেষ অংশের গতিবিধি রেকর্ড করে। 	output: যেরকম রোমে অয়্যে আর যিয়ান চেয়ে সিয়ান বানা জাহাজ আনোর হুরে হারে বেক্কানি সেজ গুরি লবঙে। 
input:যতক্ষণ পর্যন্ত না সেই সন্ত্রাসী গ্রেপ্তার না হয় । হ্যা। 	output: যেদক্কন সং না তে ব্যক্তির ১০০০ না অয় । 
input:এখন আমার কাছে সবকিছু স্পষ্ট হতে লাগল । আমার যাত্রা শেষের পথে ছিল। 	output: এখন মর ইদু বেক্কানি স্পষ্ট অবার পিরে পিয়ং হিজু ইন্দি সা বব। 
input:আমি Mr আর্লের সঙ্গে দেখা করতে এসেছি। 	output: মুই mr ওয়েন। 
input:তুমি আর আমি । 	output: তুই আর মুই । 
input:অনেক ব্যথা করছে আচ্ছা। 	output: বজমান পিরে পাত্তে আচ্ছা। 
input:অনেকগুলো তথ্য সঠিক ছিল। 	output: ভালাক্কানি হবর ঠিক এল। 


In [ ]:
input = "কোথায় চলে গিয়েছিলে?"
output = decode_sequence(input).replace("[start]","")
output = output.replace("[end]","")
print("input:" + input,"\toutput:" + output)

input:কোথায় চলে গিয়েছিলে? 	output: হুদু জাগোই । 


In [12]:
transformer.save_weights("/content/en_to_bn.h5")